# Summary

<p class='lead'>
This notebook converts a 48-spot (i.e. 96 channels) DCR file from the raw **.DAT** format saved from the LabVIEW acquisition software
into the **[Photon-HDF5](http://photon-hdf5.org)** format.
</p>

# User Input

In [1]:
fname = 'E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.dat'

In [2]:
description = 'DCR of N5 (donor) and N2 (acceptor) mantas.'

author = 'Maya Segal'
author_affiliation = 'UCLA'
creator = 'Antonino Ingargiola'
creator_affiliation = 'UCLA'

# Imports

In [3]:
import time
import numpy as np
import tables
from tqdm import tqdm_notebook, tnrange  # progress bar

import phconvert as phc
phc.__version__

'0.8'

In [4]:
import niconverter as nic
nic.__version__

'0.2'

# Preprocess spot data

In [5]:
t_start = time.time()

In [6]:
h5file, meta = nic.ni96ch_process_spots(fname, close=True)
ts_unit = 1 / meta['clock_frequency']
fname, meta

Widget Javascript not detected.  It may not be installed or enabled properly.


('E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.dat',
 {'acquisition_duration': 11067.738268187499,
  'clock_frequency': 80000000.0,
  'format': 'x',
  'nchannels': 96,
  'num_timestamps': 567861615,
  'offset': 65,
  'words_per_photon': 1})

In [7]:
time.time() - t_start

217.97720050811768

In [8]:
h5file = tables.open_file(fname[:-3] + 'hdf5', mode='a')
dt, endianess, meta = nic.detectformat(fname)
ts_unit = 1 / meta['clock_frequency']
t2, aem2 = nic.get_photon_data_arr(h5file, spots=np.arange(48))
meta['acquisition_duration'] = nic.duration(t2, ts_unit)
fname, meta

('E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.dat',
 {'acquisition_duration': 11067.738268187499,
  'clock_frequency': 80000000.0,
  'format': 'x',
  'nchannels': 96,
  'num_timestamps': 567861615,
  'offset': 65,
  'words_per_photon': 1})

In [9]:
time.time() - t_start

218.69627213478088

In [10]:
print('Measurement duration: %.1f s' %  meta['acquisition_duration'])
print('Transfer rate: %.2f MB/s' % (meta['num_timestamps'] / meta['acquisition_duration'] * 4 / (1024**2)))

Measurement duration: 11067.7 s
Transfer rate: 0.20 MB/s


In [11]:
neg_jumps = []
for tx in tqdm_notebook(t2):
    d = np.diff(tx[:])
    assert (d >= 0).all()
    neg_jumps.append(d[d<0])
    
missed_ov_jumps = [len(j) for j in neg_jumps]
for ch, jumps in enumerate(missed_ov_jumps):
    if jumps > 0:
        print('CH%d: %s' % (ch, jumps))

Widget Javascript not detected.  It may not be installed or enabled properly.


In [12]:
time.time() - t_start

235.6769700050354

# Create Photon-HDF5

In [13]:
metadata = dict(
    description = description,
    acquisition_duration = meta['acquisition_duration'],

    identity = dict(
        author=author,
        author_affiliation=author_affiliation,
        creator=creator,
        creator_affiliation=creator_affiliation),
    )
metadata

{'acquisition_duration': 11067.738268187499,
 'description': 'DCR of N5 (donor) and N2 (acceptor) mantas.',
 'identity': {'author': 'Maya Segal',
  'author_affiliation': 'UCLA',
  'creator': 'Antonino Ingargiola',
  'creator_affiliation': 'UCLA'}}

In [14]:
fname

'E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.dat'

In [15]:
h5file.filename

'E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.hdf5'

In [16]:
data = nic.create_ph5data_smFRET_48spots(fname, h5file, ts_unit, metadata)

In [17]:
[f for f in data.keys() if not f.startswith('photon')]

['_filename',
 'provenance',
 'description',
 'acquisition_duration',
 'identity',
 'setup']

In [18]:
phc.hdf5.save_photon_hdf5(data, h5file=h5file, overwrite=False, close=True)

Saving: E:/Data/Antonio/data/manta/DCR/2017-10-16/00_DCR.hdf5
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data0". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data1". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data2". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data3". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data4". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data5". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data6". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data7". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data8". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data9". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data10". 
Photon-HDF5 WARNING: Missing field "measurement_specs" in "/photon_data11". 
Photon-HDF5 WARNING: Mis

In [19]:
time.time() - t_start

257.8871908187866